In [42]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, output_notebook, show

output_notebook()
#combined first 3 columns into one date field, read in military times as strings, delayed and cancelled columns as bools 
#I'd like to turn the day_of_week column into strings with the actual days so it's easier to read (4 = Thurs)
#there are 5 digit values in the airport code columns that I need to change to the three-letter codes
#flights.groupby(flights.ORIGIN_AIRPORT).size()

Loading BokehJS ...

In [ ]:
flights=pd.read_csv('flights.csv', parse_dates=[[0,1,2]], encoding='utf-8', dtype={'ORIGIN_AIRPORT':str, 'DESTINATION_AIRPORT':str, 'SCHEDULED_DEPARTURE':str, 'DEPARTURE_TIME':str, 'WHEELS_OFF':str, 'WHEELS_ON':str, 'SCHEDULED_ARRIVAL':str, 'ARRIVAL_TIME':str, 'DIVERTED':bool, 'CANCELLED':bool})

In [98]:
#fix 5-digit airport codes so all codes are 3-digit
three_digits=pd.read_csv('threedig_airport_codes.csv') #imported with columns Code and Description
five_digits=pd.read_csv('fivedig_airport_codes.csv') #imported with columns Code and Description
df_aircodes = pd.merge(three_digits, five_digits, on='Description') #so we can merge the tables on Description
#df_aircodes.head()
#make a dictionary for easy replacement
aircode_dict = dict(zip(df_aircodes['Code_y'].astype(str),df_aircodes['Code_x']))

#flights.groupby(flights.ORIGIN_AIRPORT).size()

,YEAR_MONTH_DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
4871718,2015-10-31,6,UA,352,N494UA,12264,BOS,2225,2217,-8.0,...,2333,-22.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
4871720,2015-10-31,6,DL,1792,N809DN,14679,ATL,2230,2217,-13.0,...,0423,-4.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
4871730,2015-10-31,6,UA,1659,N448UA,12264,RDU,2230,2218,-12.0,...,2314,-21.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
4871737,2015-10-31,6,UA,554,N64809,12264,DEN,2235,2229,-6.0,...,0018,-15.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
4871748,2015-10-31,6,UA,240,N38454,14679,ORD,2243,2243,0.0,...,0327,-25.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN


In [99]:
#replace 5-digit codes with 3-digit codes (this takes a very long time to run)
#looping over the whole flight df takes too long, we can shortcut a bit if the bad rows are grouped together
#fiv_flights = flights[flights.ORIGIN_AIRPORT.str.len() == 5]
#can use these to find range values for our loop
#fiv_flights.head() #first index is at 4385950
#fiv_flights.tail() #last index is at 4871748
for i in range(4385949, 4871749):
    if i % 100000 == 0: #use this if statement to give a sense of how much work is left
        print(i)
    if len(flights['ORIGIN_AIRPORT'][i]) != 3:
        to_replace = flights['ORIGIN_AIRPORT'][i]
        value = aircode_dict[flights['ORIGIN_AIRPORT'][i]]
        flights = flights.replace(to_replace, value)
    elif len(flights['DESTINATION_AIRPORT'][i]) != 3:
        to_replace = flights['DESTINATION_AIRPORT'][i]
        value = aircode_dict[flights['DESTINATION_AIRPORT'][i]]
        flights = flights.replace(to_replace, value)



4400000
4500000
4600000
4700000
4800000


In [102]:
#test with flights.groupby(flights.ORIGIN_AIRPORT).size() to make sure it completed correctly
#this works even though it doesn't show all codes because strings with numbers sort to the top
#make sure to check DEPARTURE_AIRPORT as well
#write to csv so we don't have to run the above loop everytime
flights.to_csv('flights_clean.csv')


In [103]:
#day of week dictionary
flights = flights.replace({'DAY_OF_WEEK':{1:'Mon', 2:'Tue', 3:'Wed', 4:'Thu', 5:'Fri', 6:'Sat', 7:'Sun'}})
flights.head()

,YEAR_MONTH_DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015-01-01,Thu,AS,98,N407AS,ANC,SEA,0005,2354,-11.0,...,0408,-22.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-01,Thu,AA,2336,N3KUAA,LAX,PBI,0010,0002,-8.0,...,0741,-9.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-01-01,Thu,US,840,N171US,SFO,CLT,0020,0018,-2.0,...,0811,5.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-01-01,Thu,AA,258,N3HYAA,LAX,MIA,0020,0015,-5.0,...,0756,-9.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-01-01,Thu,AS,135,N527AS,SEA,ANC,0025,0024,-1.0,...,0259,-21.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
lga_rdu_mq = flights[(flights.ORIGIN_AIRPORT == 'LGA') & (flights.DESTINATION_AIRPORT == 'RDU') & (flights.AIRLINE == 'MQ')]
y_mq = lga_rdu_mq.DEPARTURE_DELAY
x_mq = lga_rdu_mq.YEAR_MONTH_DAY
lga_rdu_ev = flights[(flights.ORIGIN_AIRPORT == 'LGA') & (flights.DESTINATION_AIRPORT == 'RDU') & (flights.AIRLINE == 'EV')]
y_ev = lga_rdu_ev.DEPARTURE_DELAY
x_ev = lga_rdu_ev.YEAR_MONTH_DAY

In [106]:
p = figure(width=700, height=500, x_axis_type="datetime")
p.circle(x_mq, y_mq, size=7, color="firebrick", alpha=0.5)
p.circle(x_ev, y_ev, size=7, color="navy", alpha=0.5)
p.#ColumnSourceData(flights)
show(p)

AttributeError: 'Figure' object has no attribute 'ColumnSourceData'